<a href="https://colab.research.google.com/github/vladimiralencar/Alunos-UEPB-BancoDeDados/blob/master/GPU/minist-02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Sep  9 14:29:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# memória RAM em GB
!free -g

              total        used        free      shared  buff/cache   available
Mem:             25           3          19           0           3          22
Swap:             0           0           0


In [16]:
# memória RAM 
!free -h

              total        used        free      shared  buff/cache   available
Mem:            25G        3.5G         18G         14M        3.1G         22G
Swap:            0B          0B          0B


In [21]:
%%time
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 256 #128
num_classes = 10
epochs = 100 # 30  # 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
dr1 = 0.25 #0.25
model.add(Dropout(dr1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))

#model.add(Dense(256, activation='relu'))

dr2 = 0.25 # 0.25
model.add(Dropout(dr2))

#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])



model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/100
235/235 [==============================] - 1s 6ms/step - loss: 0.2214 - accuracy: 0.9324 - val_loss: 0.0521 - val_accuracy: 0.9821
Epoch 2/100
235/235 [==============================] - 1s 6ms/step - loss: 0.0622 - accuracy: 0.9811 - val_loss: 0.0551 - val_accuracy: 0.9815
Epoch 3/100
235/235 [==============================] - 1s 6ms/step - loss: 0.0432 - accuracy: 0.9864 - val_loss: 0.0360 - val_accuracy: 0.9875
Epoch 4/100
235/235 [==============================] - 1s 6ms/step - loss: 0.0329 - accuracy: 0.9895 - val_loss: 0.0333 - val_accuracy: 0.9898
Epoch 5/100
235/235 [==============================] - 1s 6ms/step - loss: 0.0267 - accuracy: 0.9913 - val_loss: 0.0430 - val_accuracy: 0.9865
Epoch 6/100
235/235 [==============================] - 1s 6ms/step - loss: 0.0224 - accuracy: 0.9930 - val_loss: 0.0295 - val_accuracy: 0.9901
Epoch 7/100
235/235 [==============================] - 1s 6ms/step - 